In [ ]:
!nvidia-smi

Mon Dec 21 00:34:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvidia-smi

Sun Dec 27 19:46:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Prepare colab environment

In [ ]:
!cd /content && export CREDENTIALS=`cat credentials.txt` && git clone https://$CREDENTIALS@github.com/SMAPWr/project-medical_sma

Cloning into 'project-medical_sma'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (325/325), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 325 (delta 178), reused 275 (delta 130), pack-reused 0
Receiving objects: 100% (325/325), 13.95 MiB | 27.52 MiB/s, done.
Resolving deltas: 100% (178/178), done.


In [ ]:
!cd /content/project-medical_sma/ && git pull

Already up to date.


In [ ]:
#!cd /content/project-medical_sma/ && git add dataset/* && git commit -m 'medical_twiteer_embeddings'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys, os
os.chdir('/content/project-medical_sma')
print(os.getcwd())
if os.getcwd() not in sys.path:
  sys.path.insert(0, os.getcwd())
print(sys.path)

/content/project-medical_sma
['/content/project-medical_sma', '', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']


In [ ]:
!date
!cp /content/drive/MyDrive/colab-data/sma-medical/*.npz dataset/
!date
!ls -l dataset/*npz

Sun Dec 27 19:47:24 UTC 2020
Sun Dec 27 19:50:09 UTC 2020
-rw-r--r-- 1 root root    5693825 Dec 27 19:46 dataset/clean_dataset_embeddings.npz
-rw------- 1 root root  928880121 Dec 27 19:47 dataset/medical_twiteer_embeddings.npz
-rw------- 1 root root 2934926103 Dec 27 19:50 dataset/znany_lekarz_2k_embeddings.npz


# Prepare dependencies

In [ ]:
!python --version
!pip install transformers

Python 3.6.9
     |████████████████████████████████| 1.5MB 8.3MB/s 
     |████████████████████████████████| 2.9MB 27.4MB/s 
     |████████████████████████████████| 890kB 54.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=04eefe968a7ddbd0ba650d922b8f1e575f6c4ae2d46c45d9839ab3e7f108a447
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import transformers
import torch
import tensorflow as tf

In [ ]:
#sanity check
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
model = AutoModel.from_pretrained("allegro/herbert-base-cased")

output = model(
    **tokenizer.batch_encode_plus(
        [
            (
                "A potem szedł środkiem drogi w kurzawie, bo zamiatał nogami, ślepy dziad prowadzony przez tłustego kundla na sznurku.",
                "A potem leciał od lasu chłopak z butelką, ale ten ujrzawszy księdza przy drodze okrążył go z dala i biegł na przełaj pól do karczmy."
            )
        ],
    padding='longest',
    add_special_tokens=True,
    return_tensors='pt'
    )
)

# Prepare embeddings


In [ ]:
#!python model/mkembeddings.py --batch 32 --word dataset/clean_dataset.csv clean_dataset_embeddings.npz 

2020-12-20 22:51:04.379480: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Tokenization completed!
100% 63/63 [00:26<00:00,  2.37it/s]


In [ ]:
!cp dataset/medical_twiteer_embeddings.npz /content/drive/MyDrive/colab-data/sma-medical/

In [ ]:
!head -n 2000 dataset/znany_lekarz_clean_dataset.csv > dataset/znany_lekarz_2k.csv

In [ ]:
!python model/mkembeddings.py --batch 16 --word dataset/znany_lekarz_2k.csv dataset/znany_lekarz_2k_embeddings.npz

2020-12-21 00:20:29.853296: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Tokenization completed!
100% 125/125 [01:36<00:00,  1.30it/s]
tcmalloc: large alloc 3156443136 bytes == 0x7fdabddc8000 @  0x7fdcb7a6d1e7 0x7fdcb399b41e 0x7fdcb39ebbdb 0x7fdcb39ebc78 0x7fdcb3a92e10 0x7fdcb3a9353c 0x7fdcb3a9365d 0x566f73 0x59fd0e 0x7fdcb39d8ea7 0x50a12f 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x588d41 0x59fd0e 0x7fdcb39d8ea7 0x50a12f 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x50ad03 0x634e72 0x634f27 0x6386df


In [ ]:
!cp dataset/znany_lekarz_2k_embeddings.npz /content/drive/MyDrive/colab-data/sma-medical/

In [ ]:
!python model/mkembeddings.py --batch 8 --word dataset/znany_lekarz_clean_dataset.csv dataset/znany_lekarz_clean_dataset_embeddings.npz

2020-12-21 00:11:25.719812: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Tokenization completed!
 48% 624/1287 [04:59<05:24,  2.04it/s]^C


In [ ]:
!cp dataset/znany_lekarz_clean_dataset_embeddings.npz /content/drive/MyDrive/colab-data/sma-medical/

cp: cannot stat 'dataset/znany_lekarz_clean_dataset_embeddings.npz': No such file or directory


In [ ]:
!nvidia-smi

Mon Dec 21 00:09:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    29W /  70W |   2471MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python model/mkembeddings.py --batch 32 --word dataset/medical_twiteer.csv dataset/medical_twiteer_embeddings.npz 

2020-12-20 22:57:27.192942: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Tokenization completed!
100% 62/62 [00:29<00:00,  2.11it/s]


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

loaded = np.load('dataset/medical_twiteer_embeddings.npz')
X = loaded['embeddings']
y = loaded['labels']
label_mappings = loaded['label_mappings']


In [ ]:
X.shape

(1958, 166, 768)

In [ ]:
y.shape

(1958,)

In [ ]:
type(y[0])

numpy.int64

In [ ]:
label_mappings

array(['Medyczne', 'Medyczne-diagnoza', 'Niemedyczny'], dtype='<U17')

In [ ]:
from dataset import loader
X, y, label_mappings = loader.load()

In [ ]:
X_train.shape, X_test.shape

((1311, 166, 768), (647, 166, 768))

In [ ]:
label_mappings

array(['Medyczne', 'Medyczne-diagnoza', 'Niemedyczny'], dtype='<U17')